
# Incrementality Test design and analysis

This notebook demonstrates a workflow for designing and implementing a Geo-based incrementality test. The notebook uses murray as a core framework, while integrating features on top of it.

In addition to the features found in murray, this notebook includes:
- EDA analysis to find low variance and high zero Geos.
- Defines experiment budget and expected CPIC in order to integrate a budget feasibility study.
- Calculates l2 imbalance in order to understand better the setup's robustness.
- Ranks and presents as output the setups based on their sensitivity, error scores, and required investment.


## 1) Experiment Design
### 1- Imports and Data Cleaning

In [ ]:
import pandas as pd
import numpy  as np
from Murray import cleaned_data
from Murray import plot_geodata,plot_impact_graphs_evaluation,plot_metrics,plot_permutation_test
from Murray import print_weights, print_incremental_results,print_incremental_results_evaluation
from Murray import run_geo_analysis,run_geo_evaluation
from Murray.plots import plot_impact_streamlit_app


In [ ]:
data = pd.read_csv("dataset2.csv")

#data = cleaned_data(data,col_target='Conversions',col_locations='Region',col_dates='Date') # for dataset1
data = cleaned_data(data,col_target='sales',col_locations='region',col_dates='date')        # for dataset2

display(data.head())

c:\Users\zmour\Desktop\Cassandra\cassandra\Lib\site-packages\Murray\auxiliary.py:49: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,time,location,Y,orders,total_sessions,total_carts,total_checkouts,total_pageviews
0,2023-08-01,quintana roo,19458.99,3.0,733.0,18.0,18.0,1853.0
1,2023-08-01,colima,699.99,1.0,188.0,8.0,4.0,466.0
2,2023-08-01,tamaulipas,2889.58,3.0,1103.0,22.0,10.0,2744.0
3,2023-08-01,durango,0.00,NaN,NaN,NaN,NaN,NaN
4,2023-08-01,nayarit,1699.99,1.0,229.0,9.0,3.0,595.0


______
### 2- EDA and Budget Definition

In [ ]:
def identify_problematic_regions(
    data: pd.DataFrame,
    value_col: str = 'Y',
    location_col: str = 'location',
    threshold_zero_ratio: float = 0.80,
    threshold_var: float = 0.1,
    ) -> list:
    """
    This is an EDA function to ensure that our data is aligned with best practices defined in Geolift, 
    and identify regions with very low variance or high ratio of zero values to recommends them for exclusion later.
    
    These regions would make it difficult to detect statistically significant variations, 
    and can produce biased control groups, making metrics such as scaledL2Imbalance untrustworthy.

    Parameters:
    - data: The input data containing the time series data.
    - value_col: The name of the column containing the target KPI.
    - location_col: The name of the column identifying geographic regions.
    - threshold_zero_ratio: The max allowed ratio of zero values in a region.
    - threshold_var: The max allowed variance in a region.

    Returns:
    - List: Regions recommended for exclusion.
    """

    
    if pd.to_datetime(data['time']).nunique() < 90:
        print("Warning: Dataset contains less than 3 months of data.")
    
    if data[location_col].nunique() < 20:
        print("Warning: Dataset contains fewer than 20 unique geo regions.")
    
    region_group = data.groupby(location_col)[value_col]

    # Identify regions with variance less than our threshold
    no_variance_regions = region_group.std()[region_group.var() < threshold_var].index.tolist()
    
    # Identify regions with % of zeros greater than our threshold
    zero_ratio = region_group.apply(lambda x: (x == 0).mean())
    high_zero_regions = zero_ratio[zero_ratio > threshold_zero_ratio].index.tolist()

    recommended_exclusions = list(set(no_variance_regions + high_zero_regions))
    print("Recommended Regions for Exclusion:")
    for region in recommended_exclusions:
        reasons = []
        if region in no_variance_regions:
            reasons.append(f"Variance lower than {threshold_var}")
        if region in high_zero_regions:
            reasons.append(f"More than {int(threshold_zero_ratio * 100)}% zero values")
        print(f" - {region}: {' and '.join(reasons)}")
    if not recommended_exclusions:
        print(" - No regions recommended for exclusion.")

    return recommended_exclusions

def define_budget_and_roi(
    experiment_budget: float,
    cost_per_incremental_conversion: float   ) -> dict:
    """
    This function simply stores the max budget and CPIC for later use in experiment planning and evaluation.   
    This is used to calculate the expected investment for the experiment, based on the approach in Geolift. 
    We will see later that the Lower the Minimum Detectable Effect (MDE), the less investment required for the experiment.

    Parameters:
    - experiment_budget (float): The actual total monthly budget available to run an incrementality test experiment.
    - cost_per_incremental_conversion (float): The cost to generate one additional conversion (CPIC), usually from our MMM model.

    Returns:
    - dict: A dictionary containing the defined parameters.
    """
    return {
        "experiment_budget": experiment_budget,
        "cost_per_incremental_conversion": cost_per_incremental_conversion}

In [138]:
problematic_regions = identify_problematic_regions(data, threshold_zero_ratio=0.25, threshold_var=0.1)

budget_inputs = define_budget_and_roi(experiment_budget=10000, cost_per_incremental_conversion=1.5)

plot_geodata(data)

Recommended Regions for Exclusion:
 - durango: More than 25% zero values
 - nayarit: More than 25% zero values
 - tlaxcala: More than 25% zero values
 - zacatecas: More than 25% zero values
 - baja california sur: More than 25% zero values
 - colima: More than 25% zero values


____

### 3- Market Selection

In [131]:
geo_test = run_geo_analysis(
    data = data,
    excluded_locations = problematic_regions,
    maximum_treatment_percentage=0.35,
    significance_level = 0.10,
    deltas_range = (0.01, 0.5, 0.02),
    periods_range = (5, 40, 5))

Simulation in progress........
Complete.


The output of the market selection function will be very useful for us below. 
In addition to the heatmap above, the run_geo_analysis outputs a dictionary of dictionaries. 

At the first level, the three dictionaries are:

- Simulation results: This contains metrics and details on the best markets for each of the group sizes (Essentially a view on each row in our heatmap).
- Sensitivity results: This contains the raw test result data behind each MDE value in the heatmap. 
- Series Lifts: This contains raw data behind the simulation showing what our market's time series would look like if a lift of size delta was applied.

In [ ]:
schema = {
    "simulation_results": {
        "group_size":                                                     # Size of the Treatment Group
            {                                          
            "Best Treatment Group":         list[str],                    # Selected Treatment Group
            "Control Group":                list[str],                    # Selected Control Group                     
            "MAPE":                         float,                        # Mean Absolute Percentage Error                     
            "SMAPE":                        float,                        # Symmetric Mean Absolute Percentage Error
            "Actual Target Metric (y)":     np.ndarray,                   # Actual Target Metric as found in the dataset
            "Predictions":                  np.ndarray,                   # Generated counterfactual predictions
            "Weights":                      np.ndarray,                   # The weights used in calculated counterfactual predictions                                    
            "Holdout Percentage":           float or None,                # 100% - Treatment Percentage     
            }
    },
    
    "sensitivity_results": {
        "group_size":                                                     # Size of the Treatment Group
            {"period":                                                    # Treatment cycle length
                {
                "Statistical Power":                                      # Dictionary of effect sizes and whether an effect was detected with power 1
                    list[
                        tuple[
                             "effect size": float,                        # Each delta value in our deltas_range
                             "power"      : float,                        # = 1 if an effect of that size was detected with power 1, else 0
                             ]
                        ],        
                "MDE":                      float or None                # The smallest effect size that was detected previously with power 1
                }
            }
    },
    
    "series_lifts": {
        "(group_size, delta, period)":      np.ndarray,                   # The timeseries of the simulated lifts for a certain setup   
    }
}


_____
### 4- Market Ranking

Below are multiple functions that come together in order to get a clearer picture of what the outputs of run_geo_analysis() are, and how to choose between them.

The approach when ranking our experiments will be trying to select a high power and low MDE experiment, while focusing on making sure that we have low error metrics. Having high error scores along side a high power would give us a powerful but biased experiment, since our control group never really matched our treatment group well. Our end goal is selecting the groups most likely to give us an experiment that is successful in measuring lift, but also making sure that we can trust the end result because it was well designed. 

The approach we went for is applying a dense ranking to our Power, MDE, Error scores, and L2 imbalance values, followed by a weighted average of all ranks. We allocated a 60% weight to sensitivity with a 40% weight to error ranking. This approach took inspriation from Geolift to apply a dense ranking.

In [139]:
def calculate_sensitivity(group_size, sensitivity_results):
    """
    Select the treatment period with the highest average power for a given group size, and return relevant sensitivity metrics.
    Our sensitivity score considers any power over 0.9 as 1.0, since usually 0.9 is a good enough threshold for power. This allows us
    to then give more focus on other metrics such as Error score, in cases where power is already satisfactory.
    
    Parameters:
    - group_size: The group size for which to calculate sensitivity metrics.
    - Sensitivity_results: The sensitivity results dictionary containing statistical power and MDE data.

    Returns:
    - best_avg_power: The best average power found for that group size.
    - best_period: The period associated with the best average power.
    - best_mde: The MDE associated with the best period.
    - sensitivity_score: A ranking score based on the average power, normalized to a maximum of 1.0.
    """
    candidate_sens = sensitivity_results.get(group_size, {})
    best_avg_power, best_period, best_mde = 0, None, None

    # Loop through each period and calculate the average power, storing the highest one.
    for period, sens_data in candidate_sens.items():
        power_list = sens_data.get("Statistical Power", [])
        if power_list:
            avg_power = np.mean([p for _, p in power_list])
            if avg_power > best_avg_power:
                best_avg_power, best_period, best_mde = avg_power, period, sens_data.get("MDE", None)

    sensitivity_score = 1.0 if best_avg_power >= 0.9 else best_avg_power / 0.9
    return best_avg_power, best_period, best_mde, sensitivity_score

def calculate_required_investment(sim_result, best_mde):
    """
    Calculate the required investment using Geolift's approach. 
    This can be compared against our budget later on to see if we can afford the experiment.
    
    Parameters:
    - sim_result: The simulation result dictionary for the candidate.
    - best_mde: The best minimum detectable effect (MDE) for the candidate.
    
    Returns:
    - Minimum required investment amount to run the experiment and achieve the MDE, considering a given CPIC.
    """
    cpic = budget_inputs.get("cost_per_incremental_conversion")
    
    baseline_conversions = np.sum(sim_result.get("Actual Target Metric (y)", []))
    if best_mde is None:
        return None
    return round(cpic * baseline_conversions * best_mde, 0)

def calculate_scaled_l2_imbalance(sim_result):
    """
    Calculate the scaled L2 imbalance using the full pre-treatment series. It is calculated as 
    the L2 distance between the treatment and control groups, divided by the L2 norm of the actual 
    target metric. This is a measure of how close the distribution of the treatment and control groups are.
    
    Parameters:
    - sim_result: The simulation result dictionary for the candidate.
    
    Returns:
    - Scaled L2 imbalance value.
    """
    y_actual = np.array(sim_result.get("Actual Target Metric (y)", []))
    y_pred = np.array(sim_result.get("Predictions", []))
    
    if y_actual.size == 0 or y_pred.size == 0:
        return np.nan
    norm_actual = np.linalg.norm(y_actual)
    return np.nan if norm_actual == 0 else np.linalg.norm(y_actual - y_pred) / norm_actual

def compute_experiment_scores(group_size, sim_result, sensitivity_results):
    """
    Compute and group multiple scores and metrics for a given experiment configuration.
    
    Parameters:
    - group_size: The group size for which to compute scores.
    - sim_result: The simulation result dictionary for the group size.
    - sensitivity_results: The sensitivity results dictionary for the group size.
    
    Returns:
    - A dictionary containing the computed scores and metrics for the configuration.
    """
    # Get the treatment and control groups from the simulation result.
    treatment_group = sim_result.get("Best Treatment Group", [])
    control_group = sim_result.get("Control Group", [])
    
    # Get the MAPE and SMAPE from the simulation result, and store the average as our error score.
    mape, smape = sim_result.get("MAPE", 0), sim_result.get("SMAPE", 0)
    error_score = (mape + smape) / 2
    
    # Get the sensitivity outputs for our group size.
    best_avg_power, best_period, best_mde, sensitivity_score = calculate_sensitivity(group_size, sensitivity_results)
    
    # Calculate the Treatment Percentage knowing the given groups.
    holdout = sim_result.get("Holdout Percentage", None)
    treatment_percentage = 100 - holdout if holdout is not None else None
    
    # Calculates ATT using a built-in murray function. 
    # Average treatment effect on the treated (ATT) is basically the average lift we expect each day during treatment.
    att_value, _, _ = plot_impact_streamlit_app(geo_test, period=best_period, holdout_percentage=holdout)
    
    # Calculate the scaled L2 imbalance between the treatment and control groups.
    scaled_l2 = calculate_scaled_l2_imbalance(sim_result)
    
    # Calculate the required investment for the experiment.
    required_investment = float(calculate_required_investment(sim_result, best_mde))
    
    return {
        "Group Size":            group_size,
        "Treatment Group":       ", ".join(treatment_group),
        "Control Group":         ", ".join(control_group),
        "Treatment Percentage":  round(treatment_percentage, 2),
        "Treatment Length":      round(best_period, 2),
        "MDE":                   best_mde,
        "Power":                 round(best_avg_power, 2),
        "Sensitivity Score":     sensitivity_score,
        "AvgATT":                round(att_value, 2),
        "SMAPE":                 round(smape, 2),
        "MAPE":                  round(mape, 2),
        "Scaled L2 Imbalance":   round(scaled_l2, 3),
        "Error Score":           round(error_score, 2),        
        "Required Investment":   required_investment
    }

def rank_experiment_results_table(geo_test_output):
    """
    The final function to rank our experiments using metrics defined above.
    
    Parameters:
    - geo_test_output: The output dictionary from the run_geo_analysis() function.
    
    Returns:
    - A DataFrame containing the top 10 ranked experiment configurations based on the computed scores.
    """
    # Extract the simulation and sensitivity results from the output of run_geo_analysis().
    simulation_results  = geo_test_output.get("simulation_results")
    sensitivity_results = geo_test_output.get("sensitivity_results")

    if simulation_results is None or sensitivity_results is None:
        raise ValueError("Output must include both simulation_results and sensitivity_results.")
    
    # Compute scores and metrics for each experiment configuration, and store them in a dataframe to apply rankings.
    configurations = [compute_experiment_scores(group_size, sim_result, sensitivity_results) for group_size, sim_result in simulation_results.items()]
    df = pd.DataFrame(configurations)

    # We compute dense ranks for each sub-score, this would allow us to rank experiments without normalizing all scores.
    df["sensitivity_rank"] = df["Sensitivity Score"  ].rank(method="dense", ascending=False)         # Higher is better.
    df["error_rank"      ] = df["Error Score"        ].rank(method="dense", ascending=True)          # Lower  is better.
    df["scaled_l2_rank"  ] = df["Scaled L2 Imbalance"].rank(method="dense", ascending=True)          # Lower  is better.
    df["mde_rank"        ] = df["MDE"                ].rank(method="dense", ascending=True)          # Lower  is better.                    

    # Total weight = 2 (sensitivity) + 1 (error) + 1 (scaled L2) + 1 (mde) = 5.
    df["avg_rank"] = (2*df["sensitivity_rank"] + df["error_rank"] + df["scaled_l2_rank"] + df["mde_rank"])/5
    df["Score"]    = (1 / df["avg_rank"]).round(2)

    # Apply a -25% penalty if required investment exceeds the experiment budget.
    df.loc[(df["Required Investment"] > budget_inputs.get("experiment_budget")), "Score"] *= 0.75

    # Add an Above/Below Budget column by comparing required investment with max budget
    df["Budget Difference"] = budget_inputs.get("experiment_budget") - df["Required Investment"]
    df["Budget Status"] = df["Budget Difference"].apply(lambda x: "Within Budget" if pd.notna(x) and x >= 0 else "Above Budget")

    # Sort and order oru final DataFrame.
    final_cols = [
        "Group Size", "Treatment Group", "Control Group", "MDE", "Treatment Length",
        "Power", "Treatment Percentage", "AvgATT", "SMAPE", "MAPE", "Scaled L2 Imbalance",
        "Required Investment", "Budget Status", "Score"]
    
    df = df.sort_values(by="Score", ascending=False).reset_index(drop=True)[final_cols].head(10)
    display(df)
    return df 

best_experiment = rank_experiment_results_table(geo_test)


,Group Size,Treatment Group,Control Group,MDE,Treatment Length,Power,Treatment Percentage,AvgATT,SMAPE,MAPE,Scaled L2 Imbalance,Required Investment,Budget Status,Score
0,13,"aguascalientes, baja california, campeche, chi...","tabasco, méxico, jalisco",0.05,20,0.92,34.35,36813.18,30.01,30.16,0.326,3332929.0,Above Budget,0.4200
1,11,"aguascalientes, chiapas, chihuahua, coahuila, ...","guanajuato, tabasco, méxico, jalisco",0.01,20,1.00,31.78,33927.21,31.02,30.93,0.331,616731.0,Above Budget,0.3750
2,15,"aguascalientes, baja california, campeche, chi...","guanajuato, tabasco, méxico, jalisco",0.11,10,0.80,33.41,48008.59,31.23,31.83,0.317,7131258.0,Above Budget,0.3150
3,14,"aguascalientes, baja california, campeche, chi...","guanajuato, tabasco, méxico, jalisco",0.11,20,0.80,33.83,32954.04,32.16,31.63,0.316,7220977.0,Above Budget,0.2700
4,12,"chiapas, coahuila, guerrero, hidalgo, nuevo le...","guanajuato, tabasco, méxico, jalisco",0.13,35,0.76,33.88,28158.67,32.22,31.28,0.322,8546397.0,Above Budget,0.2100
5,8,"chiapas, coahuila, guanajuato, guerrero, micho...","tabasco, méxico, jalisco",0.01,20,1.00,27.17,32527.72,37.14,33.53,0.375,527181.0,Above Budget,0.1725
6,10,"aguascalientes, baja california, coahuila, gue...","guanajuato, méxico, jalisco",0.15,10,0.72,30.31,44132.68,31.60,31.58,0.336,8820598.0,Above Budget,0.1650
7,6,"aguascalientes, coahuila, guerrero, jalisco, n...","guanajuato, méxico",0.01,35,1.00,27.92,22858.71,36.27,35.48,0.392,541651.0,Above Budget,0.1575
8,9,"aguascalientes, coahuila, guerrero, michoacán,...","guanajuato, méxico, jalisco",0.15,10,0.72,29.38,43255.32,31.81,32.05,0.340,8552409.0,Above Budget,0.1425
9,16,"aguascalientes, baja california, campeche, chi...","guanajuato, tabasco, méxico, jalisco, yucatán",0.15,20,0.72,33.52,34872.75,34.00,32.59,0.340,9757041.0,Above Budget,0.1350


____
### 5- Summary and Output

In [134]:
def print_candidate_summary(df):
    """
    Prints a summary for the top candidate (first row) of the ranking DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the ranking results.
    Returns:
    - Prints the summary of the top candidate.
    """

    row = df.iloc[0]
    best_treatment_percentage = row["Treatment Percentage"]
    best_period = row["Treatment Length"]
    
    print("Top Candidate Summary")
    print("-----------------")
    print(f"Group Size: {row['Group Size']}")
    print(f"Treatment Group: {row['Treatment Group']}")
    print(f"Control Group: {row['Control Group']}\n")
    
    print("Test Metrics:")
    print(f"  MDE: {round(row['MDE'],2)}")
    print(f"  Treatment Length: {row['Treatment Length']}")
    print(f"  Power: {row['Power']:.2f}")
    print(f"  Treatment Percentage: {row['Treatment Percentage']}")
    print(f"  AvgATT: {row['AvgATT']}\n")
    
    print("Error Metrics:")
    print(f"  SMAPE: {row['SMAPE']}")
    print(f"  MAPE: {row['MAPE']}\n")
    
    print("Budget Details:")
    print(f"  Required Investment: {row['Required Investment']}")
    print(f"  Budget Status: {row['Budget Status']}\n")
    
    print(f"Composite Score: {row['Score']:.4f}\n")

    
    print_incremental_results(geo_test=geo_test, period=best_period, treatment_percentage=best_treatment_percentage)

print_candidate_summary(best_experiment)

print_weights(geo_test, treatment_percentage=best_experiment["Treatment Percentage"].values[0])


Top Candidate Summary
-----------------
Group Size: 13
Treatment Group: aguascalientes, baja california, campeche, chiapas, guerrero, hidalgo, nuevo león, puebla, querétaro, sonora, tamaulipas, veracruz, yucatán
Control Group: tabasco, méxico, jalisco

Test Metrics:
  MDE: 0.05
  Treatment Length: 20
  Power: 0.92
  Treatment Percentage: 34.35
  AvgATT: 36813.18

Error Metrics:
  SMAPE: 30.01
  MAPE: 30.16

Budget Details:
  Required Investment: 16664644.0
  Budget Status: Above Budget

Composite Score: 0.4200

     Incremental Results      
ATT: 36813.18
Lift total: 736263.59


,Control Location,Weights
0,méxico,0.401307
1,jalisco,0.332766
2,tabasco,0.265927


_____
## 2) Experiment evaluation

___
### 1- Data Cleaning

In [ ]:
data = pd.read_csv("dataset2.csv")
data = cleaned_data(data,col_target='sales',col_locations='region',col_dates='date')

plot_geodata(data)

c:\Users\zmour\Desktop\Cassandra\cassandra\Lib\site-packages\Murray\auxiliary.py:49: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



________
### 2- Experiment Evaluation

In [ ]:
results = run_geo_evaluation(
     data,start_treatment='01-08-2024',
     end_treatment='20-08-2024',
     treatment_group=['aguascalientes', 'chiapas', 'guanajuato', 'guerrero', 'hidalgo', 'nuevo león', 'veracruz', 'yucatán'],
     spend=500000
     )

_____
### 3- Outcome summary and graphs

In [144]:
plot_impact_graphs_evaluation(results)

In [143]:
print_incremental_results_evaluation(results_evaluation=results,metric='CPIC')
plot_permutation_test(results_evaluation=results)

     Incremental Results      
ATT: 32495.71
Lift total: 649914.23
CPIC: 1.3


____
## Final remarks

Experiment design is a very deep rabbit hole to dive into, with many different approaches to study, each with its advantages and disadvantages. Geolift, for example, has very interesting deep dives into power curves. 

Another interesting topic that I think might be worth looking into is confidence sequences. These are especially useful in A/B testing, with continuous daily data like in our case. Confidence sequences are any-time valid which means that they are not dependent on a fixed or pre-known sample size, and could allow for early detection and stopping if a sizeable effect can be seen though our confidence sequences. More on that here: https://arxiv.org/abs/2210.08639